# Notebook principal

Sistema de predicción espacio temporal de altos riesgos delictivos

# Limpieza datos ECU911

limpieza_de_datos_ecu_911

In [ ]:
import os
import glob
import pandas as pd

# -----------------------------
# CONFIGURACIÓN
# -----------------------------
RUTA_PADRE = "../data"
RUTA_ECU911_RAW = os.path.join(RUTA_PADRE, "raw", "ecu911", "dataset")
PATRON_CSV = os.path.join(RUTA_ECU911_RAW, "*.csv")

SALIDA_LIMPIA = os.path.join(RUTA_PADRE, "interim", "ecu911", "ecu911_limpio.csv")
os.makedirs(os.path.dirname(SALIDA_LIMPIA), exist_ok=True)

archivos_csv = glob.glob(PATRON_CSV)
print(f"Archivos CSV encontrados: {len(archivos_csv)}")
print("Ejemplo:", archivos_csv[:3])

In [ ]:
def normalizar_columnas (df: pd.DataFrame):
  df = df.copy()
  df.columns = (
    df.columns.astype(str)
    .str.replace("ï»¿", "", regex=False)
    .str.strip()
    .str.lower()
  )
  return df

def limpiar_texto_basico(serie: pd.Series) -> pd.Series:
  s = serie.astype(str).str.strip()
  s = s.replace({"": pd.NA, "nan": pd.NA, "none": pd.NA, "null": pd.NA})
  return s

def estandarizar_cod_parroquia(serie: pd.Series) -> pd.Series:
  s = limpiar_texto_basico(serie)
  s = s.str.replace(".0", "", regex=False)
  s = s.str.zfill(6)
  return s

def parsear_fecha(serie: pd.Series) -> pd.Series:
    # Convierte a datetime de forma robusta
    return pd.to_datetime(serie, errors="coerce", dayfirst=True)


def limpiar_lat_lon(df: pd.DataFrame, col_lat="lat", col_lon="lon") -> pd.DataFrame:
    """
    Limpia lat/lon si existen: convierte coma->punto, numérico, elimina nulos,
    elimina ceros y rangos inválidos.
    """
    df = df.copy()

    if col_lat not in df.columns or col_lon not in df.columns:
        print(f"Aviso: no existen columnas {col_lat}/{col_lon}. Se omite limpieza lat/lon.")
        return df

    for col in [col_lat, col_lon]:
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()
            .str.replace(",", ".", regex=False)
        )
        df[col] = pd.to_numeric(df[col], errors="coerce")

    antes = len(df)
    df = df.dropna(subset=[col_lat, col_lon])

    # Quitar ceros (como en tu ejemplo)
    df = df[(df[col_lat] != 0) & (df[col_lon] != 0)]

    # Rangos válidos geográficos
    df = df[(df[col_lat].between(-90, 90)) & (df[col_lon].between(-180, 180))]

    print(f"Limpieza lat/lon: eliminados {antes - len(df)} registros inválidos.")
    return df


def leer_csv_ecu911(path_csv: str) -> pd.DataFrame:
    """
    Lee CSV ECU911 con separador ';', normaliza columnas y aplica filtro
    (si aplica) sin crear features.
    """
    df = pd.read_csv(
        path_csv,
        sep=";",
        encoding="UTF-8",
        on_bad_lines="skip",
        low_memory=False
    )
    df = normalizar_columnas(df)
    return df



In [ ]:
dfs = []
errores = []

print("Cargando archivos ECU911... esto puede tardar unos segundos...")

for archivo in archivos_csv:
    try:
        df_tmp = leer_csv_ecu911(archivo)
        # df_tmp["__source_file"] = os.path.basename(archivo)  # trazabilidad (opcional)
        dfs.append(df_tmp)
    except Exception as e:
        errores.append((archivo, str(e)))

if errores:
    print(f"Archivos con error: {len(errores)}")
    print("Ejemplo:", errores[0])

df_911 = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

print(f"Registros totales unificados: {len(df_911)}")
print(f"Columnas: {list(df_911.columns)}")
df_911.head()

In [ ]:
# -----------------------------
# 1) FILTRO OPCIONAL (si forma parte de limpieza por alcance)
# -----------------------------
if "servicio" in df_911.columns:
    antes = len(df_911)
    df_911["servicio"] = limpiar_texto_basico(df_911["servicio"])
    df_911 = df_911[df_911["servicio"] == "Seguridad Ciudadana"]
    print(f"Filtro servicio=Seguridad Ciudadana: eliminados {antes - len(df_911)}")

# -----------------------------
# 2) LIMPIEZA DE FECHA
# -----------------------------
if "fecha" in df_911.columns:
    # Parsear fecha original
    df_911["fecha_dt"] = parsear_fecha(df_911["fecha"])
    
    antes = len(df_911)
    df_911 = df_911.dropna(subset=["fecha_dt"])
    print(f"Fechas inválidas eliminadas: {antes - len(df_911)}")
    
    # Eliminar columna original y renombrar
    df_911 = df_911.drop(columns=["fecha"])
    df_911 = df_911.rename(columns={"fecha_dt": "fecha"})
    
    # Asegurar tipo datetime
    df_911["fecha"] = pd.to_datetime(df_911["fecha"])
else:
    print("Aviso: no existe columna 'fecha'.")


# -----------------------------
# 3) LIMPIEZA DE COD_PARROQUIA (solo formato, sin cruce)
# -----------------------------
# Nota: tras normalizar columnas, normalmente quedará 'cod_parroquia' aunque viniera como 'Cod_Parroquia'
if "cod_parroquia" in df_911.columns:
    df_911["cod_parroquia"] = estandarizar_cod_parroquia(df_911["cod_parroquia"])
    antes = len(df_911)
    df_911 = df_911.dropna(subset=["cod_parroquia"])
    print(f"Códigos de parroquia vacíos eliminados: {antes - len(df_911)}")
else:
    # Esto es útil para diagnosticar cómo se llama realmente
    posibles = [c for c in df_911.columns if "parroq" in c or "parro" in c]
    print("Aviso: no existe 'cod_parroquia'. Candidatas:", posibles)



In [ ]:
# Intenta limpiar lat/lon con nombres típicos: lat/lon
df_911 = limpiar_lat_lon(df_911, col_lat="lat", col_lon="lon")

# Si tu dataset usa "latitud"/"longitud" en vez de lat/lon, usa esto:
# df_911 = limpiar_lat_lon(df_911, col_lat="latitud", col_lon="longitud")

print(f"Registros después de limpieza lat/lon (si aplicó): {len(df_911)}")


In [ ]:
try:
    df_911.to_csv(SALIDA_LIMPIA, index=False, encoding="utf-8")
    print("Dataset limpio guardado correctamente.")
    print("Salida:", SALIDA_LIMPIA)
    print("Registros finales:", len(df_911))
except Exception as e:
    print("Error guardando el CSV:", e)


In [ ]:
dfs = []
errores = []

print("Cargando archivos ECU911... esto puede tardar unos segundos...")

for archivo in archivos_csv:
    try:
        df_tmp = leer_csv_ecu911(archivo)
        # df_tmp["__source_file"] = os.path.basename(archivo)  # trazabilidad (opcional)
        dfs.append(df_tmp)
    except Exception as e:
        errores.append((archivo, str(e)))

if errores:
    print(f"Archivos con error: {len(errores)}")
    print("Ejemplo:", errores[0])

df_911 = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

print(f"Registros totales unificados: {len(df_911)}")
print(f"Columnas: {list(df_911.columns)}")
df_911.head()

# ECU911 preprocesamiento con coords

preprocesamiento_datos_ecu911

In [ ]:
import os
import pandas as pd

RUTA_PADRE = "../data"

RUTA_LIMPIO = os.path.join(RUTA_PADRE, "interim", "ecu911", "ecu911_limpio.csv")
RUTA_CATALOGO = os.path.join(RUTA_PADRE, "raw", "catalogo_parroquias_ecuador.csv")

SALIDA_PREPRO = os.path.join(RUTA_PADRE, "processed", "ecu911", "ecu911_con_coords.csv")
os.makedirs(os.path.dirname(SALIDA_PREPRO), exist_ok=True)

print("Ruta limpio:", RUTA_LIMPIO)
print("Ruta catálogo:", RUTA_CATALOGO)
print("Salida:", SALIDA_PREPRO)

In [ ]:
def normalizar_columnas(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns.astype(str)
        .str.replace("ï»¿", "", regex=False)
        .str.strip()
        .str.lower()
    )
    return df

def estandarizar_cod_parroquia(serie: pd.Series) -> pd.Series:
    s = serie.astype(str).str.strip()
    s = s.str.replace(".0", "", regex=False)
    s = s.str.zfill(6)
    s = s.replace({"nan": pd.NA, "none": pd.NA, "": pd.NA})
    return s


In [ ]:
try:
    print("Cargando dataset limpio...")
    df_911 = pd.read_csv(RUTA_LIMPIO, low_memory=False)
    df_911 = normalizar_columnas(df_911)
    print("Registros:", len(df_911))
    print("Columnas:", list(df_911.columns))
except Exception as e:
    raise RuntimeError(f"Error cargando dataset limpio: {e}")


In [ ]:
try:
    print("Cargando catálogo de parroquias...")
    catalogo = pd.read_csv(RUTA_CATALOGO, dtype={"cod_parroquia": str}, low_memory=False)
    catalogo = normalizar_columnas(catalogo)
    print("Filas catálogo:", len(catalogo))
    print("Columnas catálogo:", list(catalogo.columns))
except Exception as e:
    raise RuntimeError(f"Error cargando catálogo: {e}")


In [ ]:
# Validar que exista cod_parroquia en ambos
if "cod_parroquia" not in df_911.columns:
    raise ValueError("El dataset limpio no tiene 'cod_parroquia'. Revisa nombres de columnas.")

if "cod_parroquia" not in catalogo.columns:
    raise ValueError("El catálogo no tiene 'cod_parroquia'. Revisa nombres de columnas.")

# Estandarizar
df_911["cod_parroquia"] = estandarizar_cod_parroquia(df_911["cod_parroquia"])
catalogo["cod_parroquia"] = estandarizar_cod_parroquia(catalogo["cod_parroquia"])

# Validar lat/lon
# Si en tu catálogo se llaman distinto, cámbialos aquí.
lat_col = "lat"
lon_col = "lon"

if lat_col not in catalogo.columns or lon_col not in catalogo.columns:
    raise ValueError(f"El catálogo no tiene columnas '{lat_col}'/'{lon_col}'. Columnas disponibles: {list(catalogo.columns)}")

# Mantener solo lo necesario del catálogo para evitar columnas repetidas
catalogo_coords = catalogo[["cod_parroquia", lat_col, lon_col]].drop_duplicates(subset=["cod_parroquia"])
print("Catálogo coords (únicos por cod_parroquia):", len(catalogo_coords))


In [ ]:
antes = len(df_911)

df_merge = df_911.merge(
    catalogo_coords,
    on="cod_parroquia",
    how="left"
)

print("Registros tras merge:", len(df_merge), "(debería ser igual a antes:", antes, ")")


In [ ]:
total = len(df_merge)
sin_latlon = df_merge[lat_col].isna().sum()

print("Total registros:", total)
print("Sin coordenadas:", sin_latlon)
print("Con coordenadas:", total - sin_latlon)
print("Match rate:", f"{(total - sin_latlon)/total*100:.2f}%")

# Top parroquias sin match (para depuración)
top_sin_match = (
    df_merge[df_merge[lat_col].isna()]
    .groupby("cod_parroquia")
    .size()
    .sort_values(ascending=False)
    .head(15)
)

print("\nTop cod_parroquia sin match (conteo de llamadas):")
print(top_sin_match)


In [ ]:
df_merge = df_merge.sort_values(by="fecha", ascending=True)
df_merge = df_merge.reset_index(drop=True)


In [ ]:
# Opcional: quedarte solo con filas con coordenadas
df_final = df_merge.dropna(subset=[lat_col, lon_col]).copy()

df_final.to_csv(SALIDA_PREPRO, index=False, encoding="utf-8")
print("Guardado:", SALIDA_PREPRO)
print("Registros finales con coords:", len(df_final))


# ECU911 subtipos frecuentes

exploracion_ecu_911

In [ ]:
import os
import pandas as pd

RUTA_PADRE = "../data"

RUTA_LIMPIO = os.path.join(RUTA_PADRE, "processed", "ecu911", "ecu911_con_coords.csv")


SALIDA_CONSULTA = os.path.join(RUTA_PADRE, "processed", "ecu911", "ecu911_subtipos_frecuentes.csv")
os.makedirs(os.path.dirname(SALIDA_CONSULTA), exist_ok=True)

print("Ruta limpio:", RUTA_LIMPIO)

print("Salida:", SALIDA_CONSULTA)

In [ ]:
df_ecu911 = pd.read_csv(RUTA_LIMPIO)
subtipos_frecuentes = df_ecu911['subtipo'].value_counts().reset_index()
subtipos_frecuentes.columns = ['subtipo', 'frecuencia']

subtipos_frecuentes.to_csv(SALIDA_CONSULTA, index=False)

# Preprocesamiento aprehendidos/detenidos

preprocesamiento_detenidosaprehendidos

In [ ]:
import pandas as pd
import numpy as np

#carga de datos
nombre_archivo = '../data/raw/aprehendidos_detenidos/dataset/mdi_detenidosaprehendidos_pm_2025_enero_octubre.xlsx'

In [ ]:
try:
    print("Cargando archivo Excel... esto puede tardar unos segundos...")
    df_apre = pd.read_excel(
        nombre_archivo,
        sheet_name=1,
        dtype={'codigo_parroquia': str},  # proteger columna
        engine='openpyxl'
    )
    print(f"¡Cargado! Registros totales: {len(df_apre)}")

except Exception as e:
    print("Error cargando el Excel:", e)

In [ ]:
# normalización de datos de latitud y longitud
for col in ["latitud", "longitud"]:
    df_apre[col] = (
        df_apre[col]
        .astype(str)
        .str.replace(",", ".", regex=False)
    )
    df_apre[col] = pd.to_numeric(df_apre[col], errors="coerce")

# quitar filas sin coordenadas válidas
df_apre = df_apre.dropna(subset=["latitud", "longitud"])
df_apre = df_apre[(df_apre["latitud"] != 0) & (df_apre["longitud"] != 0)]

In [ ]:
#nromalizar fecha y hora de detención/aprehensión
try:
    df_apre["fecha"] = pd.to_datetime(df_apre["fecha_detencion_aprehension"], errors="coerce")

    # hora como string limpio
    df_apre["hora_limpia"] = (
        df_apre["hora_detencion_aprehension"]
        .astype(str)
        .str.replace(" ", "")
        .str.strip()
    )

    # Combinar fecha + hora
    df_apre["fecha_completa"] = pd.to_datetime(
        df_apre["fecha"].astype(str) + " " + df_apre["hora_limpia"],
        errors="coerce"
    )

except Exception as e:
    print("Advertencia procesando fecha/hora:", e)

In [ ]:
#estandarizar el codd parroquia
if "codigo_parroquia" in df_apre.columns:
    df_apre["codigo_parroquia"] = (
        df_apre["codigo_parroquia"]
        .astype(str)
        .str.replace(r"\.0$", "", regex=True)
        .str.zfill(6)
    )


#columnas de interes
cols_interes = [
    "fecha_completa", "fecha", "latitud", "longitud",
    "codigo_parroquia", "nombre_parroquia",
    "presunta_infraccion", "tipo", "arma", "movilizacion"
]

df_apre_clean = df_apre[cols_interes].copy()

# features temporales
df_apre_clean["franja_horaria"] = df_apre_clean["fecha_completa"].dt.hour
df_apre_clean["dia"] = df_apre_clean["fecha_completa"].dt.day
df_apre_clean["mes"] = df_apre_clean["fecha_completa"].dt.month
df_apre_clean["dia_semana"] = df_apre_clean["fecha_completa"].dt.dayofweek  # 0=Lunes

#grid espacial
df_apre_clean["lat_grid"] = df_apre_clean["latitud"].round(3)
df_apre_clean["lon_grid"] = df_apre_clean["longitud"].round(3)


#conteo de delios por dia y zona
grouped = (
    df_apre_clean
    .groupby(["lat_grid", "lon_grid", "fecha"])
    .size()
    .reset_index(name="conteo_delitos")
)

df_apre_clean = df_apre_clean.merge(
    grouped,
    on=["lat_grid", "lon_grid", "fecha"],
    how="left"
)


#guardar dataset limpio
print("\n=== Dataset final limpio ===")
print(df_apre_clean.head())
print(f"Total registros finales: {len(df_apre_clean)}")

In [ ]:


print("--- Primeras Filas---")
print(df_apre_clean.head())
print("\n--- Tipos de datos ---")
print(df_apre_clean.dtypes)

In [ ]:
# 2. Validación Geográfica Rápida
print("--- Rango de Coordenadas ---")
print(f"Latitud: {df_apre_clean['latitud'].min()} a {df_apre_clean['latitud'].max()}")
print(f"Longitud: {df_apre_clean['longitud'].min()} a {df_apre_clean['longitud'].max()}")


In [ ]:

# 3. Delitos más comunes
print("\n--- Top 10 Infracciones ---")
print(df_apre_clean['presunta_infraccion'].value_counts().head(10))

In [ ]:
# ============================================================
# CLASIFICACIÓN DE DELITOS GRAVES
# ============================================================

delitos_interes = [
    'DELITOS CONTRA EL DERECHO A LA PROPIEDAD',
    'DELITOS POR LA PRODUCCIÓN O TRÁFICO ILÍCITO DE SUSTANCIAS CATALOGADAS SUJETAS A FISCALIZACIÓN',
    'DELITOS CONTRA LA SEGURIDAD PÚBLICA',
    'DELITOS CONTRA LA EFICIENCIA DE LA ADMINISTRACIÓN PÚBLICA',
    'DELITOS DE VIOLENCIA CONTRA LA MUJER O MIEMBROS DEL NÚCLEO FAMILIAR'
]

# 1. Etiqueta de delito grave (0/1)
df_apre_clean["es_delito_grave"] = df_apre_clean["presunta_infraccion"].isin(delitos_interes).astype(int)

# 2. Conteo solo de delitos graves (target alternativo)
df_graves = (
    df_apre_clean[df_apre_clean["es_delito_grave"] == 1]
    .groupby(["lat_grid", "lon_grid", "fecha"])
    .size()
    .reset_index(name="conteo_delitos_graves")
)

# Asegurar que no exista antes de unir
if "conteo_delitos_graves" in df_apre_clean.columns:
    df_apre_clean.drop(columns=["conteo_delitos_graves"], inplace=True)

# Merge limpio
df_apre_clean = df_apre_clean.merge(
    df_graves,
    on=["lat_grid", "lon_grid", "fecha"],
    how="left"
)

# Rellenar NaN con 0 (ningún delito grave en esa celda y día)
df_apre_clean["conteo_delitos_graves"] = df_apre_clean["conteo_delitos_graves"].fillna(0)

In [ ]:
# ============================================================
# LIMPIEZA DE POSIBLES DUPLICADOS (col_x, col_y)
# ============================================================

cols_a_borrar = [c for c in df_apre_clean.columns if c.endswith("_x") or c.endswith("_y")]

if len(cols_a_borrar) > 0:
    print(f"\nEliminando columnas duplicadas generadas por merge: {cols_a_borrar}")
    df_apre_clean.drop(columns=cols_a_borrar, inplace=True)


print("\n--- Distribución de delitos graves (solo para análisis) ---")
print(df_apre_clean["es_delito_grave"].value_counts())

In [ ]:
# ============================================================
# GUARDADO (solo CSV)
# ============================================================

df_apre_clean.to_csv("../data/interim/aprehendidos_Detenidos/aprehendidos_limpio_final.csv", index=False)

print("\n¡Archivo 'aprehendidos_limpio_final.csv' guardado con éxito!")
print(f"Registros finales: {len(df_apre_clean)}")

# Preprocesamiento conteo dias conteo

preprocesamiento_ecu911

## VOLVER A CORREGIR LO DE VOLVER A CONCATENAR LOS ARCHIVOS DE ECU911

In [ ]:
import pandas as pd
import glob
import os

ruta_carpeta_911 = "../data/raw/ecu911/dataset/"
ruta_catalogo = "../data/processed/"
ruta_dest = "../data/processed/ecu911"

archivos_csv = glob.glob(os.path.join(ruta_carpeta_911, "*.csv"))

In [ ]:
lista_dfs = []

for archivo in archivos_csv:
    df_temp = pd.read_csv(
        archivo,
        sep=";",
        encoding="UTF-8",
        dtype={"Cod_Parroquia": str},
        on_bad_lines="skip"
    )

    df_temp.columns = (
        df_temp.columns.astype(str)
        .str.replace("ï»¿", "")
        .str.strip()
        .str.lower()
    )

    if "servicio" in df_temp.columns:
        df_temp = df_temp[df_temp["servicio"] == "Seguridad Ciudadana"]

    lista_dfs.append(df_temp)

In [ ]:
df_911 = pd.concat(lista_dfs, ignore_index=True)

In [ ]:
# Fecha
df_911["fecha_dt"] = pd.to_datetime(df_911["fecha"], errors="coerce", dayfirst=True)
df_911 = df_911.dropna(subset=["fecha_dt"])


In [ ]:
# Normalizar cod_parroquia
df_911["cod_parroquia"] = (
    df_911["cod_parroquia"]
    .astype(str)
    .str.replace(".0", "", regex=False)
    .str.zfill(6)
)

In [ ]:
# (CLAVE) agrupar por parroquia y día (si quieres día, normaliza a date)
df_911["fecha_dia"] = df_911["fecha_dt"].dt.date

df_group = (
    df_911.groupby(["cod_parroquia", "fecha_dia"])
    .size()
    .reset_index(name="conteo_llamadas_riesgo")
)

In [ ]:
# Catálogo: asegurar 1 fila por cod_parroquia antes de merge
catalogo = pd.read_csv(
    os.path.join(ruta_catalogo, "catalogo_parroquias_ecuador.csv"),
    dtype={"cod_parroquia": str}
)

catalogo["cod_parroquia"] = catalogo["cod_parroquia"].astype(str).str.zfill(6)

# si hay repetidos, quédate con uno (o define una regla mejor si aplica)
catalogo = catalogo.drop_duplicates(subset=["cod_parroquia"], keep="first")

In [ ]:
# Merge para lat/lon (ahora NO explota)
df_final = df_group.merge(catalogo[["cod_parroquia", "lat", "lon"]], on="cod_parroquia", how="left")


In [ ]:
# Eliminar sin coords si quieres
df_final = df_final.dropna(subset=["lat", "lon"])

#grid espacial
df_final["lat_grid"] = df_final["lat"].round(3)
df_final["lon_grid"] = df_final["lon"].round(3)

In [ ]:
# Features temporales sobre fecha_dia
df_final["fecha_dia"] = pd.to_datetime(df_final["fecha_dia"])
df_final["mes"] = df_final["fecha_dia"].dt.month
df_final["dia"] = df_final["fecha_dia"].dt.day
df_final["dia_semana"] = df_final["fecha_dia"].dt.dayofweek

df_final.to_csv(os.path.join(ruta_dest, "ecu911_parroquia_dia_conteo.csv"), index=False)

print("ECU911 agregado por parroquia y día correctamente")
print("Registros finales:", len(df_final))